In [29]:

import time

In [5]:
import requests
import json
# import pandas as pd
# from datetime import datetime

# API Configuration
BASE_URL = "http://localhost:8001"
HEADERS = {"Content-Type": "application/json"}

def post_request(endpoint: str, payload: dict, headers: dict, timeout: int = 15):
    url = f"{BASE_URL}/{endpoint}"
    
    try:
        response = requests.post(
            url,
            json=payload,
            headers=headers,
            timeout=timeout
        )
        
        print(f"\n📡 POST to {endpoint}")
        print(f"Status Code: {response.status_code}")
        print(f"Response Time: {response.elapsed.total_seconds():.2f}s")
        
        if response.status_code == 200:
            result = response.json()
            print("✅ Request successful!")
            return result
        else:
            print(f"❌ Request failed: {response.text}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Request error: {e}")
        return None

## Product matching

In [ ]:
## Setting input
caption_payload = {
    "image_path": "Data/Image12.png"
}
user_messages = {
        "messages": [
            {"role": "user", "content": "Hey, I'm shopping on Vibe and looking for outfits for summer."},
            {"role": "assistant", "content": "Sure! Do you have a specific style or occasion in mind?"},
            {"role": "user", "content": "Yes, I'm going to a beach party and I want something casual but cute."},
            {"role": "assistant", "content": "We have floral maxi dresses and off-shoulder sundresses perfect for the beach."},
            {"role": "user", "content": "Okay, I also want something for a formal dinner."},
            {"role": "assistant", "content": "Great! Our satin slip dresses and belted midi dresses would be ideal."},
            {"role": "user", "content": "I want the following items"}
        ]
    }

In [30]:
start = time.time()

## Image Captioning request
print("image captioning...")
result = post_request("image_captioning", caption_payload, HEADERS)
image_caption = result['results']

print("find docs...")
doc_payload = {
    "user_query": f'''{user_messages}''',
    "image_prompt": image_caption
}

result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
reference = result['results']
print(reference)


print("generate response...")
payload = {
    "question": f'''{user_messages}''',
    "reference": f'''{reference}'''
}

result = post_request("generation", payload, HEADERS, timeout=30)
print(result["response"])

end = time.time()
print(f"Total time taken: {end - start:.2f} seconds")

image captioning...

📡 POST to image_captioning
Status Code: 200
Response Time: 2.85s
✅ Request successful!
find docs...

📡 POST to finding_documents
Status Code: 200
Response Time: 6.57s
✅ Request successful!
[{'search_term': 'floral maxi dress', 'search_results': [{'score': 0.7890574, 'id': 'Image40.jpg', 'name': 'Yellow Floral Spaghetti Strap Midi Dress', 'description': 'This enchanting yellow midi dress features delicate spaghetti straps and a flattering sweetheart neckline with a charming ruched or tie-front detail at the bust. Adorned with a vibrant floral print in shades of pink, purple, and orange, the dress has a flowing silhouette that gracefully falls to a midi length with an asymmetrical hem. Style Vibe: Romantic Summer / Whimsical Chic. Fit: Fitted bust with a flowing A-line midi skirt. Material: Lightweight, airy fabric (likely chiffon or georgette) with a subtle sheen. Care: Dry clean recommended or delicate hand wash. Matches Well With: Outerwear: A light white cardigan

## Style matching

In [ ]:
start = time.time()

## Image Captioning request
print("image captioning...")
result = post_request("image_captioning", caption_payload, HEADERS)
image_caption = result['results']

print("style complement...")
doc_payload = {
    "user_query": f'''{user_messages}''',
    "image_prompt": image_caption
}
result = post_request("style_complement", doc_payload, HEADERS, timeout=30)
style_complement = result['results']
print("find docs...")
doc_payload = {
    "user_query": f'''{style_complement}''',
    "image_prompt": image_caption
}

result = post_request("finding_documents", doc_payload, HEADERS, timeout=30)
reference = result['results']
print(reference)


print("generate response...")
payload = {
    "question": f'''{user_messages}''',
    "reference": f'''{reference}'''
}

result = post_request("generation", payload, HEADERS, timeout=30)
print(result["response"])

end = time.time()
print(f"Total time taken: {end - start:.2f} seconds")

In [ ]:
print("=== Testing Finding Documents (Red Dress) ===\n")

doc_payload = {
    "user_query": f'''{user_messages}''',
    "image_prompt": image_caption
}

result = post_request("style_complement", doc_payload, HEADERS, timeout=30)
style_complement = result['results']
if result:
    print(result["results"])

=== Testing Finding Documents (Red Dress) ===


📡 POST to style_complement
Status Code: 200
Response Time: 3.83s
✅ Request successful!
Based on the image of a professional-looking gray button-down dress/coat and the previous chat context about seeking summer and formal outfits, here are recommended search queries:

[
    "tailored gray shirt dress with belt",
    "professional summer midi dress with collar",
    "neutral tone button-down work dress",
    "minimalist gray office dress with waist cinch",
    "chic structured gray day-to-night dress",
    "versatile button-front work dress with belt",
    "sophisticated gray shirt dress for office",
    "elegant neutral tone midi dress with long sleeves"
]
